# Self-contained Robo-advisor

In [ ]:
import pandas as pd
import plotly.graph_objects as go

from PortfolioPredictor import evaluate_portfolios_over_time, extract_asset_returns, chain_portfolio_performance
from fetchData import fetch_raw_data_yf, getSNP500, getNasdaq_comp
from LearningRBA import MLRBA_V2
from PortfolioFunction import get_matrices

In [ ]:
assets = getSNP500()
start_date = pd.Timestamp("2021-01-01")
end_date = pd.Timestamp("2023-01-01")
investment_period = 60
investment_end_date = end_date + pd.Timedelta(days=investment_period)
raw_data, _, _= fetch_raw_data_yf(assets, start_date, investment_end_date)

In [ ]:
investments = evaluate_portfolios_over_time(raw_data, end_date, window_size=5, threshold=0.5, epochs=20, length_of_investment = None)

In [ ]:
names, annualized_returns, _, _, _, cov, correlation_matrix = get_matrices(raw_data[:end_date])
_, initial_portfolio, _, _, _ = MLRBA_V2(names, cov, annualized_returns, correlation_matrix)

In [ ]:
ML_portfolio = []
portfolio_holdings = []  # To store tickers held at each point
initial_value = 100

for i in range(len(investments)):
    curr_best_portfolio = investments[i]['portfolio']
    p_start_date = investments[i]['start_date']
    p_end_date = investments[i]['end_date']
    
    if not curr_best_portfolio:
        # No portfolio for this period
        if i == 0:
            previous_value = initial_value
        else:
            previous_value = ML_portfolio[-1].iloc[-1]
        window_index = raw_data.loc[p_start_date:p_end_date].index
        portfolio_daily_returns = pd.Series(previous_value, index=window_index)
        held_tickers = []  # Nothing held
    else:
        # Portfolio exists for this period
        best_curr_port_assets = curr_best_portfolio['tickers']
        best_curr_port_assets_test_data = extract_asset_returns(raw_data, best_curr_port_assets, p_start_date, p_end_date)
        curr_best_portfolio_weights = curr_best_portfolio['weights']
        weighted_returns = best_curr_port_assets_test_data.mul(curr_best_portfolio_weights, axis='columns')
        portfolio_daily_returns = weighted_returns.sum(axis=1)
        held_tickers = best_curr_port_assets

    ML_portfolio.append(portfolio_daily_returns)
    portfolio_holdings.append((p_start_date, p_end_date, held_tickers))

ML_portfolio_streamed = chain_portfolio_performance(ML_portfolio, starting_value=initial_value)

# Print tickers held at each point
for period_info in portfolio_holdings:
    start, end, tickers = period_info
    print(f"Held from {start} to {end}: {tickers}")

#Number of unique assets held
unique_assets = set()
for _, _, tickers in portfolio_holdings:
    unique_assets.update(tickers)

print(f"\nNumber of unique tickers held: {len(unique_assets)}")

ML_portfolio_streamed

In [ ]:
ML_daily_returns = ML_portfolio_streamed.pct_change()
ML_cumulative_returns = (1 + ML_daily_returns).cumprod()
ML_cumulative_returns.iloc[0] = 1
ML_portfolio_normalized = (ML_cumulative_returns / ML_cumulative_returns.iloc[0]) * 100

best_port_assets = initial_portfolio['tickers']
best_port_assets_test_data = raw_data.loc[:, best_port_assets]

Nasdaq_comp = getNasdaq_comp(ML_cumulative_returns.index[0], ML_cumulative_returns.index[-1])

best_portfolio_weights = initial_portfolio['weights']
normalized_prices = best_port_assets_test_data.div(best_port_assets_test_data.iloc[0])
daily_returns = normalized_prices.pct_change()
weighted_returns = daily_returns.mul(best_portfolio_weights, axis='columns')
portfolio_daily_returns = weighted_returns.sum(axis=1)
portfolio_cumulative_returns = (1 + portfolio_daily_returns).cumprod()

portfolio_start = portfolio_cumulative_returns.iloc[0]
portfolio_normalized = (portfolio_cumulative_returns / portfolio_start) * 100

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=Nasdaq_comp.index,
    y=Nasdaq_comp['Normalized'],
    mode='lines',
    name='Nasdaq Composite'
))

fig.add_trace(go.Scatter(
    x=ML_cumulative_returns.index,
    y=ML_portfolio_normalized,
    mode='lines',
    name='Portfolio with Adjustments'
))

fig.add_trace(go.Scatter(
    x=ML_cumulative_returns.index,
    y=portfolio_normalized,
    mode='lines',
    name='Base Portfolio'
))

fig.update_layout(
    title='Comparison of Portfolio vs. Nasdaq Composite Growth : 2023-2025',
    xaxis_title='Date',
    yaxis_title='Normalized Returns (Base 100%)',
    xaxis=dict(
        type='date',
        tickformat='%b %Y',
        tickmode='auto'
    ),
    font=dict(
        family="Cambria",
        size=18,
    )
)
fig.show()
#fig.write_html("MLRBAvsRBAvsNasdaq.html")
#fig.write_image("MLRBAvsRBAvsNasdaq.png", format='png', width=1920, height=1080)